In [1]:
import pandas as pd

In [2]:
# Cesty k súborom
file_control    = r"C:\Users\marti\Diplomovka\DiplomovkaOpravene.xlsx"
file_stanza     = r"C:\Users\marti\Diplomovka\stanza\outputStanza.xlsx"
file_trankit    = r"C:\Users\marti\Diplomovka\Trankit\Trankit_words.xlsx"
file_treetagger = r"C:\Users\marti\Diplomovka\TreeTagger\outputTreeTagger.xlsx"
file_udpipe     = r"C:\Users\marti\Diplomovka\UDPipe\UDPipe.xlsx"
file_morphodita = r"C:\Users\marti\Diplomovka\morphodita\words.xlsx"

In [4]:
# ------------------------------
# 1. Vytvorenie agregovaného DataFrame s tokenmi a UPOS tagmi
# ------------------------------

# Načítanie kontrolného datasetu – predpokladáme, že obsahuje stĺpce "Token" a "Xpos1"
control_df = pd.read_excel(file_control)
n = len(control_df)  # počet riadkov podľa kontrolného datasetu

aggregatedUPOS = pd.DataFrame()
aggregatedUPOS['Control_Token'] = control_df['Token'].iloc[:n].reset_index(drop=True)
aggregatedUPOS['Control_UPOS']  = control_df['Xpos1'].iloc[:n].reset_index(drop=True)

# Funkcia, ktorá načíta daný súbor a vráti stĺpce: Token, UPOS tag a ID (ak existuje)
def load_library(file_path, token_col="Token", upos_col="Xpos1", id_col="ID", n_rows=None):
    df = pd.read_excel(file_path)
    if n_rows is not None:
        df = df.iloc[:n_rows].reset_index(drop=True)
    token_series = df[token_col]
    upos_series = df[upos_col]
    id_series = df[id_col] if id_col in df.columns else None
    return token_series.reset_index(drop=True), upos_series.reset_index(drop=True), (id_series.reset_index(drop=True) if id_series is not None else None)

# Stanza (predpokladáme: stĺpce "Token", "Xpos1" a "ID")
stanza_token, stanza_upos, stanza_id = load_library(file_stanza, token_col="Token", upos_col="Xpos1", id_col="ID", n_rows=n)
aggregatedUPOS['stanza_Token'] = stanza_token
aggregatedUPOS['stanza_UPOS']  = stanza_upos
aggregatedUPOS['stanza_ID']    = stanza_id

# Trankit (predpokladáme: stĺpce "Token", "XPOS1" a "ID")
trankit_token, trankit_upos, trankit_id = load_library(file_trankit, token_col="Token", upos_col="XPOS1", id_col="ID", n_rows=n)
aggregatedUPOS['trankit_Token'] = trankit_token
aggregatedUPOS['trankit_UPOS']  = trankit_upos
aggregatedUPOS['trankit_ID']    = trankit_id

# TreeTagger (predpokladáme: stĺpce "Token", "Xpos1" a "ID")
treetagger_token, treetagger_upos, treetagger_id = load_library(file_treetagger, token_col="Token", upos_col="Xpos1", id_col="ID", n_rows=n)
aggregatedUPOS['treetagger_Token'] = treetagger_token
aggregatedUPOS['treetagger_UPOS']  = treetagger_upos
aggregatedUPOS['treetagger_ID']    = treetagger_id

# UDPipe (predpokladáme: stĺpce "Token", "Xpos1" a "ID")
udpipe_token, udpipe_upos, udpipe_id = load_library(file_udpipe, token_col="Token", upos_col="Xpos1", id_col="ID", n_rows=n)
aggregatedUPOS['udpipe_Token'] = udpipe_token
aggregatedUPOS['udpipe_UPOS']  = udpipe_upos
aggregatedUPOS['udpipe_ID']    = udpipe_id

# Morphodita – špeciálne: UPOS získame zo stĺpca "Tag" (iba prvé písmeno) a ID je v stĺpci "Sentence_ID"
df_morphodita = pd.read_excel(file_morphodita)
df_morphodita = df_morphodita.iloc[:n].reset_index(drop=True)
aggregatedUPOS['morphodita_Token'] = df_morphodita['Token']
aggregatedUPOS['morphodita_UPOS']  = df_morphodita['Tag'].astype(str).str[0]
aggregatedUPOS['morphodita_Sentence_ID'] = df_morphodita['Sentence_ID']

# Uloženie agregovaného DataFrame do Excelu
aggregatedUPOS.to_excel(r"C:\Users\marti\Diplomovka\agregovanyUPOS.xlsx", index=False)

# ------------------------------
# 2. Porovnanie UPOS tagov s kontrolou
# ------------------------------

# Pre každý zdroj vytvoríme porovnávací stĺpec, ktorý obsahuje True, ak sa UPOS tag daného zdroja rovná Control_UPOS, inak False.
sources = {
    'stanza': 'stanza_UPOS',
    'trankit': 'trankit_UPOS',
    'treetagger': 'treetagger_UPOS',
    'udpipe': 'udpipe_UPOS',
    'morphodita': 'morphodita_UPOS'
}

for lib, upos_col in sources.items():
    comp_col = f'{lib}_Match'
    aggregatedUPOS[comp_col] = aggregatedUPOS[upos_col] == aggregatedUPOS['Control_UPOS']

# Uloženie výsledného DataFrame (s porovnaním) do Excelu
aggregatedUPOS.to_excel(r"C:\Users\marti\Diplomovka\agregovanyUPOS_with_Match.xlsx", index=False)

# Výpis prvých niekoľkých riadkov na kontrolu
print(aggregatedUPOS.head())

  Control_Token Control_UPOS stanza_Token stanza_UPOS  stanza_ID  \
0            Je            V           Je           V          1   
1            už            T           už           T          2   
2         skoro            T        skoro           D          3   
3       október            S      október           S          4   
4             a            O            a           O          5   

  trankit_Token trankit_UPOS trankit_ID treetagger_Token treetagger_UPOS  ...  \
0            Je            V       None               Je               V  ...   
1            už            T       None               už               T  ...   
2         skoro            T       None            skoro               T  ...   
3       október            S       None          október               S  ...   
4             a            O       None                a               O  ...   

   udpipe_UPOS udpipe_ID morphodita_Token  morphodita_UPOS  \
0            V         1               Je 

In [7]:
from collections import Counter
Counter(aggregatedUPOS['trankit_Match'])

Counter({False: 859, True: 340})

In [ ]:
aggregatedUPOS

,Control,stanza1,trankit1,treetagger1,udpipe1,morphodita1
0,V,V,V,V,V,V
1,T,T,T,T,T,X
2,T,D,T,T,T,D
3,S,S,S,S,S,N
4,O,O,O,O,O,J
...,...,...,...,...,...,...
1194,S,S,E,S,S,V
1195,V,V,S,V,V,A
1196,E,E,V,E,E,N
1197,S,S,A,A,S,N


In [ ]:
# Predpokladáme, že match_df obsahuje stĺpce:
# "Control", "stanza1", "trankit1", "treetagger1", "udpipe1", "morphodita1"
# a že v týchto stĺpcoch (okrem Control) sú booleovské hodnoty True/False.

total_rows = len(match_df)
success_summary = {}

# Iterujeme cez všetky stĺpce okrem "Control"
for col in match_df.columns:
    if col == "Control":
        continue
    # Počet True (match) v danom stĺpci
    count_true = match_df[col].sum()  # True sa počíta ako 1
    percentage = (count_true / total_rows) * 100
    success_summary[col] = percentage

# Výpis výsledkov
print("Percentuálny podiel úspešnosti pre jednotlivé knižnice:")
for lib, pct in success_summary.items():
    print(f"{lib}: {pct:.2f}%")


Percentuálny podiel úspešnosti pre jednotlivé knižnice:
stanza1: 96.83%
trankit1: 28.36%
treetagger1: 29.11%
udpipe1: 97.75%
morphodita1: 12.76%
